In [42]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import haversine_distances
from math import radians

def combine_lat_lon(df):
    """Combine latitude and longitude into a single column."""
    df['coordinates'] = list(zip(df['latitude'], df['longitude']))
    return df

def run_kmeans(orders_df, vehicle_df):
    """Run KMeans clustering on order coordinates."""
    num_clusters = len(vehicle_df) - 1
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(list(orders_df['coordinates']))
    orders_df['cluster'] = kmeans.labels_
    return orders_df, kmeans

def print_clusters(orders_df, kmeans):
    """Print clusters."""
    cluster_ids = np.unique(kmeans.labels_)
    for cluster_id in range(len(cluster_ids)):
        cluster_orders = orders_df[orders_df['cluster'] == cluster_id]['order_id']
        print(f"Cluster {cluster_id + 1}: Orders {', '.join(cluster_orders.astype(str))}")

def plot_clusters(orders_df, filename='cluster_map.html'):
    """Plot clusters on a folium map and save it to an HTML file."""
    map_center = [np.mean(orders_df['latitude']), np.mean(orders_df['longitude'])]
    m = folium.Map(location=map_center, zoom_start=10)
    colors = ['red', 'green', 'blue', 'yellow', 'cyan', 'magenta', 'black', 'orange']
    for idx, row in orders_df.iterrows():
        folium.CircleMarker(location=row['coordinates'], color=colors[row['cluster']], fill=True, radius=5).add_to(m)
    m.save(filename)
    return m

def haversine(point1, point2):
    """Calculate haversine distance between two points."""
    point1 = [radians(coord) for coord in point1]
    point2 = [radians(coord) for coord in point2]
    distance = haversine_distances([point1, point2])
    return distance[0][1] * 6371000

def select_nearest_cluster(orders_df, leaving_point):
    """Select the nearest cluster to the leaving point."""
    orders_df['distance_to_leaving_point'] = orders_df['coordinates'].apply(lambda x: haversine(x, leaving_point))
    nearest_cluster = orders_df.loc[orders_df['distance_to_leaving_point'].idxmin()]
    return nearest_cluster

def assign_vehicle(nearest_cluster, vehicle_df):
    """Assign the first vehicle from the vehicle data file to the nearest cluster."""
    vehicle_id = vehicle_df.iloc[0]['vehicle_id']
    vehicle_capacity = vehicle_df.iloc[0]['vehicle_capacity ']
    return vehicle_id, vehicle_capacity

def collect_orders(nearest_cluster_orders, vehicle_capacity):
    """Collect orders until vehicle capacity is reached."""
    collected_orders = []
    remaining_capacity = vehicle_capacity
    for idx, order in nearest_cluster_orders.iterrows():
        if remaining_capacity >= order['order_weight']:
            current_order = order['order_id']
            current_weight = order['order_weight']
            collected_orders.append(current_order)
            remaining_capacity -= current_weight
            print(f'collected order {current_order} with weight {current_weight} . Remains {remaining_capacity}')
        else:
            exceeds_weight_order = order['order_weight']
            print(f'cant take {exceeds_weight_order}')
            break
    return collected_orders

def remove_collected_orders(orders_df, collected_orders):
    """Remove collected orders from the data."""
    orders_df = orders_df[~orders_df['order_id'].isin(collected_orders)]
    return orders_df

def remove_vehicle(vehicle_df, vehicle_id):
    """Remove vehicle orders from the data."""
    vehicle_df = vehicle_df[vehicle_df['vehicle_id']!=vehicle_id]
    return vehicle_df




iteration 1

In [43]:
# Usage example:
orders_df = pd.read_csv('data.csv')
vehicle_df = pd.read_csv('vehicle_data.csv')

orders_df = combine_lat_lon(orders_df)
orders_df, kmeans = run_kmeans(orders_df, vehicle_df)

print_clusters(orders_df, kmeans)
m = plot_clusters(orders_df)
display(m)
leaving_point = (52.499773, -2.024925)
nearest_cluster = select_nearest_cluster(orders_df, leaving_point)
vehicle_id, vehicle_capacity = assign_vehicle(nearest_cluster, vehicle_df)
nearest_cluster_orders = orders_df[orders_df['cluster'] == nearest_cluster['cluster']]
nearest_cluster_orders = nearest_cluster_orders.sort_values(by='distance_to_leaving_point')

print(f'vehicle capacity is {vehicle_capacity}')

collected_orders = collect_orders(nearest_cluster_orders, vehicle_capacity)



orders_df = remove_collected_orders(orders_df, collected_orders)

vehicle_df = remove_vehicle(vehicle_df, vehicle_id)

print("Collected Orders:", collected_orders)
print("Vehicle Assigned:", vehicle_id)

/home/gustavo/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster 1: Orders 8, 12, 14, 16, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 50, 53, 54, 55, 57, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 73, 74, 76, 77, 79, 80
Cluster 2: Orders 29, 49
Cluster 3: Orders 4, 5, 6
Cluster 4: Orders 13, 52, 60, 78
Cluster 5: Orders 10, 15, 20, 21, 51, 56, 75
Cluster 6: Orders 11, 47, 64, 72
Cluster 7: Orders 1, 2, 3, 7, 32, 48
Cluster 8: Orders 9


vehicle capacity is 600
collected order 6 with weight 155 . Remains 445
collected order 5 with weight 93 . Remains 352
collected order 4 with weight 147 . Remains 205
Collected Orders: [6, 5, 4]
Vehicle Assigned: 100


iteration 2. now without the taken orders

In [44]:
orders_df

,order_id,location,order_weight,special_care,order_type,stop_number,latitude,longitude,coordinates,cluster,distance_to_leaving_point
0,1,19 chandlers close B18 5rx,200,1,collection,1,52.493374,-1.930552,"(52.49337389999999, -1.9305519)",6,6428.226493
1,2,192a Bacchus Rd B18 4RB,150,0,collection,2,52.496314,-1.934332,"(52.4963138, -1.9343318)",6,6144.684186
2,3,44 Majuba Rd B16 0PA,37,1,collection,3,52.486321,-1.950021,"(52.48632139999999, -1.9500208)",6,5287.143277
6,7,225 Sandwell Rd B21 8PD,111,0,collection,7,52.514217,-1.947607,"(52.5142167, -1.9476072)",6,5473.818904
7,8,7 Masshouse Ln B5 5JN,52,0,delivery,8,52.481643,-1.889314,"(52.4816434, -1.8893144)",0,9400.284442
...,...,...,...,...,...,...,...,...,...,...,...
75,76,176 Edmund St B3 2HB,96,0,delivery,76,52.482997,-1.899629,"(52.4829972, -1.8996293)",0,8685.747694
76,77,2 Chamberlain Sq B3 3AX,58,0,collection,77,52.479826,-1.904839,"(52.4798262, -1.9048386)",0,8427.771106
77,78,230 Birmingham Rd B43 7AG,96,1,collection,78,52.558539,-1.938368,"(52.5585392, -1.9383678)",3,8774.048710
78,79,17 Thorp St B5 4AT,14,1,collection,79,52.474807,-1.898476,"(52.4748075, -1.8984763)",0,9000.728193


In [45]:
vehicle_df

,vehicle_id,vehicle_capacity
1,101,800
2,102,700
3,103,300
4,104,200
5,105,500
6,106,800
7,107,700
8,108,1000


In [46]:
orders_df = combine_lat_lon(orders_df)
orders_df, kmeans = run_kmeans(orders_df, vehicle_df)

print_clusters(orders_df, kmeans)
m = plot_clusters(orders_df)
display(m)
leaving_point = (52.499773, -2.024925)
nearest_cluster = select_nearest_cluster(orders_df, leaving_point)
vehicle_id, vehicle_capacity = assign_vehicle(nearest_cluster, vehicle_df)
nearest_cluster_orders = orders_df[orders_df['cluster'] == nearest_cluster['cluster']]
nearest_cluster_orders = nearest_cluster_orders.sort_values(by='distance_to_leaving_point')


print(f'vehicle capacity is {vehicle_capacity}')

collected_orders = collect_orders(nearest_cluster_orders, vehicle_capacity)
orders_df = remove_collected_orders(orders_df, collected_orders)

vehicle_df = remove_vehicle(vehicle_df, vehicle_id)

print("Collected Orders:", collected_orders)
print("Vehicle Assigned:", vehicle_id)

display(orders_df)
display(vehicle_df)



/home/gustavo/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster 1: Orders 29, 49
Cluster 2: Orders 8, 12, 14, 16, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 50, 53, 54, 55, 57, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 73, 74, 76, 77, 79, 80
Cluster 3: Orders 13, 52, 60, 78
Cluster 4: Orders 10, 15, 20, 21, 51, 56, 75
Cluster 5: Orders 1, 2, 3, 7, 32, 48
Cluster 6: Orders 9
Cluster 7: Orders 11, 47, 64, 72


vehicle capacity is 800
collected order 3 with weight 37 . Remains 763
collected order 7 with weight 111 . Remains 652
collected order 48 with weight 48 . Remains 604
collected order 2 with weight 150 . Remains 454
collected order 1 with weight 200 . Remains 254
collected order 32 with weight 169 . Remains 85
Collected Orders: [3, 7, 48, 2, 1, 32]
Vehicle Assigned: 101


,order_id,location,order_weight,special_care,order_type,stop_number,latitude,longitude,coordinates,cluster,distance_to_leaving_point
7,8,7 Masshouse Ln B5 5JN,52,0,delivery,8,52.481643,-1.889314,"(52.4816434, -1.8893144)",1,9400.284442
8,9,68 The Green B38 8RU,162,0,delivery,9,52.407110,-1.930478,"(52.4071104, -1.9304778)",5,12129.488031
9,10,22 Highfield Rd B15 3DP,146,0,collection,10,52.469281,-1.923066,"(52.4692809, -1.9230656)",3,7685.713249
10,11,4 Woodbridge Rd B13 8EJ,31,0,collection,11,52.448175,-1.887104,"(52.4481749, -1.8871041)",6,10957.024213
11,12,16 Gas St B1 2JT,189,0,collection,12,52.477502,-1.910521,"(52.4775019, -1.9105212)",1,8132.357084
...,...,...,...,...,...,...,...,...,...,...,...
75,76,176 Edmund St B3 2HB,96,0,delivery,76,52.482997,-1.899629,"(52.4829972, -1.8996293)",1,8685.747694
76,77,2 Chamberlain Sq B3 3AX,58,0,collection,77,52.479826,-1.904839,"(52.4798262, -1.9048386)",1,8427.771106
77,78,230 Birmingham Rd B43 7AG,96,1,collection,78,52.558539,-1.938368,"(52.5585392, -1.9383678)",2,8774.048710
78,79,17 Thorp St B5 4AT,14,1,collection,79,52.474807,-1.898476,"(52.4748075, -1.8984763)",1,9000.728193


,vehicle_id,vehicle_capacity
2,102,700
3,103,300
4,104,200
5,105,500
6,106,800
7,107,700
8,108,1000
